# คู่มือเตรียมพรีเซนต์โปรเจกต์ Reviewer Hub

เอกสารนี้สรุปเนื้อหาสำคัญสำหรับใช้ในการนำเสนอโปรเจกต์ โดยเน้นที่ความเข้าใจภาพรวมของระบบและ Code SQL ที่สำคัญในแต่ละส่วน

---

## 1. ภาพรวมระบบ (System Overview)

**Reviewer Hub** เป็นเว็บแอปพลิเคชันสำหรับรีวิวสินค้า โดยมีผู้ใช้งาน 3 ระดับ คือ:
1.  **Customer (ลูกค้าทั่วไป)**: ดูสินค้า, เขียนรีวิว, คอมเมนต์
2.  **Merchant (ร้านค้า)**: เหมือนลูกค้า แต่สามารถ **เพิ่มร้านค้า** และ **เพิ่มสินค้า** ของตัวเองได้
3.  **Admin (ผู้ดูแลระบบ)**: ดูแลความเรียบร้อย สามารถ **ลบ** สินค้า/รีวิว/คอมเมนต์ ที่ไม่เหมาะสม หรือ **แบน** ผู้ใช้งานได้

### Flow การทำงานหลัก
- **การเข้าใช้งาน**: เริ่มต้นทุกคนเป็น Guest (ดูได้อย่างเดียว) -> ต้อง **สมัครสมาชิก/เข้าสู่ระบบ** จึงจะใช้งานฟีเจอร์ต่างๆ ได้
- **การรีวิว**: สินค้า 1 ชิ้น -> มีหลาย **รีวิว** (จากลูกค้า) -> แต่ละรีวิว -> มีหลาย **คอมเมนต์** (พูดคุยกันใต้รีวิว)

---

## 2. เจาะลึกโค้ดและการทำงานแต่ละไฟล์ (Code & SQL Logic)

### 📁 ไฟล์: `register.php` (ระบบสมัครสมาชิก)
**หน้าที่**: รับข้อมูลจากผู้ใช้เพื่อสร้างบัญชีใหม่
**Logic สำคัญ**:
- รับค่า Username, Email, Password, และ Role (ประเภทบัญชี)
- ตรวจสอบว่า Username หรือ Email ซ้ำหรือไม่ก่อนบันทึก

**🛠 SQL ที่ใช้**:
1.  **ตรวจสอบข้อมูลซ้ำ**:
    ```sql
    SELECT user_id FROM User WHERE username = ? OR email = ?
    ```
    *(ถ้าเจอข้อมูล แปลว่าซ้ำ -> แจ้งเตือน)*

2.  **บันทึกสมาชิกใหม่**:
    ```sql
    INSERT INTO User (user_type_id, username, email, password)
    VALUES (?, ?, ?, ?)
    ```
    *(เก็บข้อมูลลงฐานข้อมูล)*

---

### 📁 ไฟล์: `login.php` (ระบบเข้าสู่ระบบ)
**หน้าที่**: ตรวจสอบตัวตนผู้ใช้เพื่อเริ่ม Session
**Logic สำคัญ**:
- เช็คว่าผู้ใช้กรอก Username หรือ Email ตรงกับในฐานข้อมูลไหม
- เช็คว่ารหัสผ่านมาตรงกันหรือไม่
- ถ้าผ่าน -> เก็บ `user_id`, `user_type_id` ลง Session

**🛠 SQL ที่ใช้**:
1.  **ดึงข้อมูลผู้ใช้**:
    ```sql
    SELECT user_id, username, email, password, user_type_id
    FROM User
    WHERE username = ? OR email = ?
    LIMIT 1
    ```

---

### 📁 ไฟล์: `index.php` (หน้าแรก/Dashboard)
**หน้าที่**: แสดงรายการสินค้าทั้งหมด มีระบบค้นหาและกรอง
**Logic สำคัญ**:
- แสดงสินค้า 50 รายการล่าสุด (หรือตามเงื่อนไขค้นหา)
- คำนวณ **Rating เฉลี่ย** ของแต่ละสินค้า
- กรองตามหมวดหมู่ (Category) หรือค้นหาชื่อ (Search)

**🛠 SQL ที่ใช้**:
1.  **ดึงสินค้า + คะแนนเฉลี่ย + ชื่อร้าน**:
    ```sql
    SELECT p.product_id, p.product_name, p.category,
           s.store_name,
           ROUND(AVG(r.rating), 1) AS avg_rating,  -- หาค่าเฉลี่ยดาว
           COUNT(r.review_id) AS review_count      -- นับจำนวนรีวิว
    FROM Product p
    JOIN Store s ON p.store_id = s.store_id        -- เชื่อมตารางร้านค้า เพื่อเอาชื่อร้าน
    LEFT JOIN Review r ON r.product_id = p.product_id -- เชื่อมรีวิว เพื่อคำนวณคะแนน
    WHERE 1=1 ... (เงื่อนไข Search/Filter)
    GROUP BY p.product_id
    ORDER BY ... 
    LIMIT 50
    ```

---

### 📁 ไฟล์: `product.php` (หน้ารายละเอียดสินค้า & รีวิว)
**หน้าที่**: แสดงข้อมูลลึกของสินค้า, แสดงรีวิว, แสดงคอมเมนต์, และฟอร์มส่งรีวิว/คอมเมนต์
**Logic สำคัญ**:
- ดึงข้อมูลรานละเอียดสินค้า และร้านค้าเจ้าของ
- ดึงรีวิวทั้งหมดของสินค้านั้น
- ในแต่ละรีวิว ดึงคอมเมนต์ที่เกี่ยวข้องมาแสดง
- (Admin) มีปุ่มลบสินค้า/แบนผู้ใช้

**🛠 SQL ที่ใช้**:
1.  **ดึงข้อมูลสินค้า**:
    ```sql
    SELECT p.*, s.store_id, s.user_id, s.store_name ...
    FROM Product p
    JOIN Store s ON p.store_id = s.store_id
    WHERE p.product_id = ?
    ```

2.  **ดึงรีวิว และคนเขียนรีวิว**:
    ```sql
    SELECT r.*, u.username, u.user_type_id
    FROM Review r
    JOIN User u ON r.user_id = u.user_id
    WHERE r.product_id = ?
    ```

3.  **เพิ่มรีวิวใหม่**:
    ```sql
    INSERT INTO Review (product_id, user_id, rating, review_text)
    VALUES (?, ?, ?, ?)
    ```

4.  **เพิ่มคอมเมนต์ใหม่**:
    ```sql
    INSERT INTO Comment (user_id, review_id, comment_text)
    VALUES (?, ?, ?)
    ```

5.  **(Admin) ลบสินค้า (ต้องลบข้อมูลที่เกี่ยวโยงกันก่อน)**:
    - ลบ Comment ที่อยู่ในรีวิวของสินค้านี้
    - ลบ Review ของสินค้านี้
    - สุดท้าย ลบ Product

---

### 📁 ไฟล์: `add-store.php` & `add-product.php` (ส่วนของ Merchant)
**หน้าที่**: ให้ร้านค้าสร้าง Profile ร้าน และลงสินค้าขาย

**🛠 SQL ที่ใช้**:
1.  **สร้างร้านค้า**:
    ```sql
    INSERT INTO Store (user_id, store_name, country, city, contact, ...)
    VALUES (?, ?, ?, ?, ?, ...)
    ```

2.  **เพิ่มสินค้า (ต้องผูกกับ Store ID)**:
    ```sql
    INSERT INTO Product (store_id, product_name, description, category)
    VALUES (?, ?, ?, ?)
    ```

---

### 📁 ไฟล์: `user-profile.php` (หน้าโปรไฟล์)
**หน้าที่**: ดูข้อมูลส่วนตัว, ประวัติรีวิว, ร้านค้าของฉัน, และแก้ไขข้อมูล

**🛠 SQL ที่ใช้**:
1.  **ดูข้อมูล User**:
    ```sql
    SELECT * FROM User WHERE user_id = ?
    ```

2.  **อัปเดตข้อมูลส่วนตัว**:
    ```sql
    UPDATE User SET username=?, email=?, password=? WHERE user_id=?
    ```

3.  **ดึงประวัติรีวิวของฉัน (Join เพื่อรู้ว่ารีวิวของอะไร)**:
    ```sql
    SELECT r.*, p.product_name, s.store_name
    FROM Review r
    JOIN Product p ON r.product_id = p.product_id
    JOIN Store s ON p.store_id = s.store_id
    WHERE r.user_id = ?
    ```

---

## สรุปปิดท้าย
ระบบ Reviews Hub ใช้หลักการ **Relational Database** เชื่อมโยงข้อมูลผ่าน ID (เช่น `product_id`, `user_id`, `store_id`) เพื่อให้ข้อมูลมีความสัมพันธ์กัน และใช้การ **JOIN** ตารางในการดึงข้อมูลมาแสดงผลหน้าเว็บให้ครบถ้วนในคำสั่งเดียวครับ
